In [31]:
import pandas as pd
from bs4 import BeautifulSoup

In [10]:
df = pd.read_json("/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthData.jsonl", orient="records", lines=True)

In [32]:
df.columns

Index(['potentialOutPath', 'transcript', 'wCount', 'rssUrl', 'epTitle',
       'epDescription', 'duration', 'pubDate', 'copyright', 'itunes:type',
       'itunes:complete', 'guid', 'itunes:explicit', 'enclosure',
       'itunes:image', 'transDict', 'id', 'title', 'lastUpdate', 'link',
       'lastHttpStatus', 'dead', 'contentType', 'itunesId', 'originalUrl',
       'itunesAuthor', 'itunesOwnerName', 'explicit', 'imageUrl', 'itunesType',
       'generator', 'newestItemPubdate', 'language', 'oldestItemPubdate',
       'episodeCount', 'popularityScore', 'priority', 'createdOn',
       'updateFrequency', 'chash', 'host', 'newestEnclosureUrl', 'podcastGuid',
       'podDescription', 'category1', 'category2', 'category3', 'category4',
       'category5', 'category6', 'category7', 'category8', 'category9',
       'category10', 'newestEnclosureDuration', 'oldestItemDatetime',
       'cleanDates', 'cleanDatesLoc', 'parsedDuration', 'hours', '500ent',
       '500start', '500end', '500type', 'Des

In [21]:
sample = df[["rssUrl", "transcript", "enclosure", "guid", "cleanDatesLoc", "epTitle", "title", "podDescription", "epDescription"]].sample(1000, random_state=120)

In [22]:
sample = sample.dropna().drop_duplicates(subset=["enclosure"])

In [33]:
#TODO: clean out html tags and stuff like that... 
DESC_LEN=100
TRANS_LEN=300

def makePresentationText(inRow): 
    description = inRow["epDescription"]
    transcript = inRow["transcript"]

    description = BeautifulSoup(description, "html.parser").text
    transcript = BeautifulSoup(transcript, "html.parser").text

    if len(description.split()) > DESC_LEN: 
        description = " ".join(description.split()[:DESC_LEN]) + "..."
    
    if len(transcript.split()) > TRANS_LEN: 
        transcript = " ".join(transcript.split()[:TRANS_LEN]) + "..."
    
    return f"Description:\n{description} \n\n Transcript:\n{transcript}"

In [35]:
sample["displayText"] = sample.apply(makePresentationText, axis=1)

/tmp/ipykernel_1725681/3483627211.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, "html.parser").text
/tmp/ipykernel_1725681/3483627211.py:9: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  description = BeautifulSoup(description, "html.parser").text


In [36]:
list(sample["displayText"])[:10]

["Description:\nWe are always in progressThe “It’s not fast enough” thoughtWhat if self sabotage isn’t a real thing?Grace with the unpredictable pace of lifeMarking progress as a tool in the gameRespect and appreciation for the unknownDoes life make it’s own progress?The dance of giving something our time and attentionSuffering from an overinflated sense of “I”\xa0Beating ourselves up with what we make upWhy does nothing\xa0require\xa0our worry? \n\n Transcript:\n[MUSIC] Hello and welcome. You're listening to Zennegan, What Do We Know? A podcast that's all about exploring the human experience from an unofficial point of view. Your hosts are Mermansen and Brienne Grebel, that's me. We hope you'll sit back, relax, and just ponder what we're talking about. As we explore new ways of looking at life, we are hoping that you will do the same. So without further ado, let's get into today's episode. [MUSIC] Hello and welcome to Zennegan, What Do We Know? I'm Brienne Grebel with me is Mermansen.

In [37]:

list(sample["epDescription"])[:10]

["<p class=''>We are always in progress</p><p class=''>The “It’s not fast enough” thought</p><p class=''>What if self sabotage isn’t a real thing?</p><p class=''>Grace with the unpredictable pace of life</p><p class=''>Marking progress as a tool in the game</p><p class=''>Respect and appreciation for the unknown</p><p class=''>Does life make it’s own progress?</p><p class=''>The dance of giving something our time and attention</p><p class=''>Suffering from an overinflated sense of “I”&nbsp;</p><p class=''>Beating ourselves up with what we make up</p><p class=''>Why does nothing&nbsp;<em>require</em>&nbsp;our worry?</p>",
 "We are doing a series called Ameri-CAN. Talking about the CAN-DO spirit that is part of our American DNA. So much about our lives has been turned upside down by this Corona Virus / Covid-19, and we have been hearing a lot about what we CAN’T do and where we CAN’T go, but our guests during this series are going to talk to us about their area of expertise and all of th

In [30]:
sample.to_json("/shared/3/projects/benlitterer/podcastData/annotation/1000sample.jsonl", orient="records", lines=True)